In [1]:
!pip install statsmodels
import pandas as pd
import numpy as np
eyes = pd.read_csv('eyes_mean.csv', header=None)
visual = pd.read_csv('visual_raw.csv', header=None)
somatomotor = pd.read_csv('somatomotor_raw.csv', header=None)
n_subjects = eyes.shape[1] 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Stationarity test (not necessary all the time)
from statsmodels.tsa.stattools import adfuller

for x in range(n_subjects):
  #Compute the stationary test, change the variable to test the other signals
  dftest = adfuller(eyes.loc[:,x], autolag="AIC")
  #Print the p-values showing <0.05 if the test of stationarity is passed
  print(dftest[1])

2.466878501686909e-06
1.6189001006928704e-07
0.002573240752109721
5.684784857242624e-16
0.0015555279180113103
8.223301269535089e-07
0.0037127393673670357
7.4364769687998974e-06
0.00015687112615603906
0.0002698324702281903
7.424053747000281e-06
3.4590879440120316e-07
0.00011749786195750658
1.583077393767153e-05


In [41]:
from statsmodels.tsa.stattools import grangercausalitytests
# Perform Granger-Causality test
max_lags = 5
causal_mat = np.zeros((3, 3))
ROIs = ['visual','somatomotor','eyes']

def find_causality(max_lags,seriesto,seriesfrom):
  pvals = []
  for x in range(n_subjects):
    print(x)
    # Check whether the first variable is caused by the second one
    df = pd.DataFrame(list(zip(seriesto.loc[:,x], seriesfrom.loc[:,x])))
    # Difference to make the signal further stationary (it can also smooth further the signal loosing information)
    #df = pd.DataFrame(list(zip(np.diff(visual.loc[:,x])[1:], np.diff(eyes.loc[:,x])[1:])))
    gc_res = grangercausalitytests(df, maxlag=max_lags)
    # Remove List and Dict
    gc_res_lags =  (gc_res[max_lags])
    unlisted_gc_res_lags = gc_res_lags[0]
    pval = unlisted_gc_res_lags['ssr_ftest'][1]
    pvals.append(pval)
    #print(pvals)
    avg_pval = np.median(pvals)
  return avg_pval,pvals

# Use a for loop to iterate over the list
j_ind = 0
for ROI_i in ROIs:
  i_ind = 0
  for ROI_j in ROIs:
     avg_pval,pvals = find_causality(max_lags, locals()[ROI_i] ,locals()[ROI_j])
     causal_mat[i_ind, j_ind] = avg_pval
     i_ind = i_ind + 1
  j_ind = j_ind + 1

print(causal_mat)
causal_mat<0.05

0

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0000  , p=1.0000  , df_denom=129, df_num=1
ssr based chi2 test:   chi2=0.0000  , p=1.0000  , df=1
likelihood ratio test: chi2=-0.0000 , p=1.0000  , df=1
parameter F test:         F=220.4737, p=0.0000  , df_denom=129, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.0000  , p=1.0000  , df_denom=127, df_num=2
ssr based chi2 test:   chi2=0.0000  , p=1.0000  , df=2
likelihood ratio test: chi2=-0.0000 , p=1.0000  , df=2
parameter F test:         F=156.5830, p=0.0000  , df_denom=127, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=-0.0000 , p=1.0000  , df_denom=125, df_num=3
ssr based chi2 test:   chi2=-0.0000 , p=1.0000  , df=3
likelihood ratio test: chi2=-0.0000 , p=1.0000  , df=3
parameter F test:         F=103.1533, p=0.0000  , df_denom=125, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=-0.0000 , p=1.0000

array([[False,  True, False],
       [False, False, False],
       [ True, False, False]])